# Data z webu EMIS

**Zdroje znečišťování za rok 2017**

Údaje o provozovnách a emisích ohlášených v souhrnné provozní evidenci za aktuální rok na webu [ČHMÚ](http://portal.chmi.cz/files/portal/docs/uoco/web_generator/plants/index_CZ.html).

Provozovna na webu EMIS nutně neznamená jeden zdroj, jak bychom si intuitivně přáli (jednu bioplynku, jednu výtopnu) apod. Viz příklad [Energetiky Kněžice](http://portal.chmi.cz/files/portal/docs/uoco/web_generator/plants/CZ020/211902152_CZ.html), kde je v jednom zdroji spojená výtopna na pevnou biomasu (kotel na štěpku a kotel na slámu) a bioplynová stanice, viz licence na webu [ERÚ](http://licence.eru.cz/detail.php?lic-id=310605027&sequence=1,2&total=2). Může jít o kombinaci více zdrojů v jednom areálu.

Zajištění dat ve struktuře jako na webu (bez rozdělení do vhodnějších kategorií, např. oddělit psč a obec) ve formátu CSV. 

Schází doplnit kolonku Paliva pro technologie a Přehled vyjmenovaných zdrojů.

## Odkazy na zdroje
Odkazy na jednotlivé zdroje znečištění. 

Web je strukturován jako stránka s odkazy na kraje, každá stránka kraje obsahuje odkazy na stránku s okresy. Až na stránce pro příslušný okres jsou odkazy na jednotlivé provozovny (kromě Prahy, kde jsou odkazy na provozovny umístěné na stránce kraje).

In [1]:
import unicodedata
import requests
from bs4 import BeautifulSoup

def get_links(url, base):
    """Return list of all links from a single webpage (table with kraje and okresy 
    on EMIS website).
    """
    r = requests.get(url)
    r.encoding = 'utf-8'
    bs = BeautifulSoup(r.text, 'html.parser')
    links = bs.find('table', {'class':'data_table'}).find_all('a')
    links = [base + link.attrs['href'] for link in links]
    return links

index = 'http://portal.chmi.cz/files/portal/docs/uoco/web_generator/plants/index_CZ.html'
base = 'http://portal.chmi.cz/files/portal/docs/uoco/web_generator/plants/'

# Get links to kraje pages
kraje_links = get_links(index, base)

# Get links to okresy pages (Praha w/o okres level)
okresy_links = [kraje_links[0]]
for kraj_link in kraje_links[1:]: # Praha w/o okres level
    okresy_links.extend(get_links(kraj_link, base))

# Get links to individual utilities (provozovny)
links = []
for okres_link in okresy_links:
    links.extend(get_links(okres_link, base))
    
print(f'Z webu EMIS vráceno {len(links)} odkazů na zdroje znečištění')

Z webu EMIS vráceno 10063 odkazů na zdroje znečištění


## Data o zdrojích

Data o jednotlivých zdrojích znečištění (provozovnách).

In [2]:
def get_data(url):
    """Return data about a single utility (provozovna) as a dictionary.
    """
    r = requests.get(url)
    r.encoding = 'utf-8'
    bs = BeautifulSoup(r.text, 'html.parser')
    table = bs.find('table', {'class':'data_table'})
    
    prov = {}
    
    prov['url'] = url
    
    # Basic data (same for each utility) are extracted using the basic dictionary
    basics = {'nazev': 'Název:',\
              'nace': 'NACE:',\
              'ulice_cp': 'Ulice, č.p./č.o.:',\
              'psc_obec':'PSČ, Obec:',\
              'souradnice': 'Zeměpisné souřadnice:',\
              'adm': 'Adresní místo (ADM):',\
              'prikon': 'Celkový příkon provozovny [MW]: '}
    for basic in basics:
        if basic == 'adm':
            try:
                prov[basic] = table.find('td', text=basics[basic]).next_sibling.next_sibling.a.get_text()
            except AttributeError:
                prov[basic] = 'n/a'
        else:
            prov[basic] = table.find('td', text=basics[basic]).next_sibling.next_sibling.get_text()
    
    # Emissions (emise) and fuels (paliva) do not have a specified number of rows
    rows = table.find_all('tr')
    for i in range(len(rows)):
        if rows[i].th:
            if rows[i].th.get_text() == 'Emise [t]':
                start_emiss = i + 1
            if rows[i].th.get_text() == 'Doplňující údaje':
                end_emiss = i            
            if rows[i].th.get_text() == 'Paliva ze spalovacích procesů':
                start_fuels = i + 1
            if rows[i].th.get_text() == 'Paliva pro technologie':
                end_fuels = i
    
    # Fuels are only a name without a value
    fuels = []
    for i in range(start_fuels, end_fuels):
        fuels.append(rows[i].td.get_text())
    # Fuels will be stored as a string, fuels separated by comma.
    prov['paliva'] = ', '.join(fuels)
    
    # Emissions consist of substance name and value 
    for i in range(start_emiss, end_emiss):
        key = rows[i].td.get_text()
        key = unicodedata.normalize('NFKD', key).encode('ascii', 'ignore')
        key = key.decode('utf-8')
        key = key.replace(' ', '_')
        try:
            value = rows[i].td.next_sibling.next_sibling.get_text()
        except AttributeError:
            break
        prov[key] = value
    return prov

In [3]:
# Test url
test_url = 'http://portal.chmi.cz/files/portal/docs/uoco/web_generator/plants/CZ010/727878391_CZ.html'
test_data = get_data(test_url)
test_data

{'url': 'http://portal.chmi.cz/files/portal/docs/uoco/web_generator/plants/CZ010/727878391_CZ.html',
 'nazev': 'ABB s.r.o. - Polovodiče',
 'nace': '271200',
 'ulice_cp': 'Novodvorská 1768/138a',
 'psc_obec': '142 21 Praha',
 'souradnice': '50° 1´ 40.854" sš 14° 25´ 55.702" vd',
 'adm': '25206257',
 'prikon': '3,8',
 'paliva': 'nafta, zemní plyn',
 'tuhe_znecistujici_latky': '0,006',
 'oxidy_dusiku': '0,332',
 'oxid_uhelnaty': '0,007',
 'tekave_organicke_latky_(VOC)': '3,419'}

In [4]:
# Extracting data from all urls
emis_data = []
params = []

for link in links:
    item = get_data(link)
    emis_data.append(item)
    params.extend([key for key in item.keys() if key not in params])
    if links.index(link) % 1000 == 0:
        print(f'Zpracováno {links.index(link)} odkazů z {len(links)}')

print(f'V databázi je celkem {len(emis_data)} provozoven (zdrojů znečištění).')

Zpracováno 0 odkazů z 10063
Zpracováno 1000 odkazů z 10063
Zpracováno 2000 odkazů z 10063
Zpracováno 3000 odkazů z 10063
Zpracováno 4000 odkazů z 10063
Zpracováno 5000 odkazů z 10063
Zpracováno 6000 odkazů z 10063
Zpracováno 7000 odkazů z 10063
Zpracováno 8000 odkazů z 10063
Zpracováno 9000 odkazů z 10063
Zpracováno 10000 odkazů z 10063
V databázi je celkem 10063 provozoven (zdrojů znečištění).


In [5]:
import csv

with open('emis_data.csv', mode='w') as csvf:
    writer = csv.writer(csvf, delimiter=',')
    writer.writerow(params)
    for util in emis_data:
        row = []
        for param in params:
            try:
                row.append(util[param])
            except KeyError:
                row.append('n/a')
        writer.writerow(row)
print('CSV zapsáno.')

CSV zapsáno.
